In [2]:
import duckdb
import pandas as pd
from IPython.display import display

pd.set_option('display.max_columns', None)

In [4]:

# Path to DuckDB database
DB_PATH = '../../database/clinisys_all.duckdb'

use_schema = 'silver'

# List of relevant silver tables
silver_tables = [
    'view_micromanipulacao',
    'view_congelamentos_ovulos',
    'view_descongelamentos_ovulos',
    'view_congelamentos_embrioes',
    'view_embrioes_congelados',
    'view_descongelamentos_embrioes',
    'view_tratamentos',
    # 'view_micromanipulacao_oocitos',
]

# Table that needs special handling
micromanip_oocitos_table = 'view_micromanipulacao_oocitos'


# Prompt for prontuario
prontuario = '876950' #'160751' #'171805'

# Connect to DuckDB
db = duckdb.connect(DB_PATH)

# Special handling for micromanipulacao_oocitos
print(f'\n--- Records from {use_schema}.{micromanip_oocitos_table} for prontuario={prontuario} (via view_micromanipulacao) ---')
try:
    # Get all codigo_ficha for this prontuario
    codigos = db.execute(f"""
        SELECT codigo_ficha FROM {use_schema}.view_micromanipulacao WHERE prontuario = ?
    """, [prontuario]).fetchall()
    codigo_list = [row[0] for row in codigos]
    if codigo_list:
        # Query oocitos table for all id_micromanipulacao in codigo_list
        placeholders = ','.join(['?']*len(codigo_list))
        query = f"""
            SELECT * FROM {use_schema}.{micromanip_oocitos_table}
            WHERE id_micromanipulacao IN ({placeholders})
            ORDER BY id_micromanipulacao, id
        """
        df = db.execute(query, codigo_list).df()
        if not df.empty:
            display(df)
        else:
            print('No records found.')
    else:
        print('No matching codigo_ficha found in view_micromanipulacao.')
except Exception as e:
    print(f'Error querying {use_schema}.{micromanip_oocitos_table}:', e)


for table in silver_tables:
    print(f'\n--- Records from {use_schema}.{table} for prontuario={prontuario} ---')
    # Some tables may not have prontuario, so try/except
    try:
        df = db.execute(f"""
            SELECT * FROM {use_schema}.{table}
            WHERE prontuario = ?
            order by 1
        """, [prontuario]).df()
        if not df.empty:
            print(f'Shape: {df.shape}')
            display(df)
        else:
            print('No records found.')
    except Exception as e:
        print(f'Error querying {use_schema}.{table}:', e)

db.close()


--- Records from silver.view_micromanipulacao_oocitos for prontuario=876950 (via view_micromanipulacao) ---


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,266484,26626,Não,MII,None,None,4318,None,Descartado,None,None,None,None,DEG,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4ec4ed0b0acb4a4c5037a478619a3f3d,2025-07-29 19:05:28,1,1
1,266485,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0a519b5d037a6abf4ea4b7ec8d811296,2025-07-29 19:05:28,1,2
2,266486,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,28a32891d80d829a1e6aa03d95ba2883,2025-07-29 19:05:28,1,3
3,266487,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,9d830272ca36bbd15c89f656d96a7c8b,2025-07-29 19:05:28,1,4
4,266488,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1880f61c2872473173371be3df9f8133,2025-07-29 19:05:28,1,5
5,266489,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6ebc8278fb48f2045246b8727d15d0f1,2025-07-29 19:05:28,1,6
6,266490,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,ea1b743a38746991ce862ad337456803,2025-07-29 19:05:28,1,7
7,266491,26626,Não,MII,None,None,4318,None,None,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,463fded6ff7e410d196861f04f9c5394,2025-07-29 19:05:28,1,8
8,266492,26626,Não,MII,None,None,4318,None,Descartado,None,None,None,None,DEG,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7d92b892df874e8b4b7c7f02ca0bbb9c,2025-07-29 19:05:28,1,9
9,266493,26626,Não,MII,None,None,4318,None,Descartado,None,None,None,None,1CP,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,557a2fc2d659b911c986e8bcc55dad7a,2025-07-29 19:05:28,1,10



--- Records from silver.view_micromanipulacao for prontuario=876950 ---
Shape: (1, 211)


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,26626,2519/25,876950,33,32,2025-07-28,None,13:50,14:10,10:45,31,3755,None,Não,None,None,None,12:25,08:00,None,None,None,None,None,22,34,None,None,None,None,None,Não,None,None,None,None,2025-07-28,ICSI,12 espermatozoides móveis progressivos\r\n,12 espermatozoides normais\r\n,None,None,None,None,None,None,0000000001,<NA>,<NA>,<NA>,3613,3492,None,Non


--- Records from silver.view_congelamentos_ovulos for prontuario=876950 ---
No records found.

--- Records from silver.view_descongelamentos_ovulos for prontuario=876950 ---
No records found.

--- Records from silver.view_congelamentos_embrioes for prontuario=876950 ---
No records found.

--- Records from silver.view_embrioes_congelados for prontuario=876950 ---
No records found.

--- Records from silver.view_descongelamentos_embrioes for prontuario=876950 ---
No records found.

--- Records from silver.view_tratamentos for prontuario=876950 ---
No records found.
